# MAD! Multivariate Alteration Detection (Part 1)


### Imports & Authentication

In [4]:
import ee

ee.Authenticate(auth_mode='notebook')
ee.Initialize(project='satvis-425514')


In [5]:
%matplotlib inline
import geemap
import numpy as np
import random, time
import matplotlib.pyplot as plt
from scipy.stats import norm, chi2

from pprint import pprint
from typing import Tuple


In [6]:
# truncate a value to 'dec' number of positions
def trunc(values, dec = 3):
    return np.trunc(values*10**dec)/(10**dec)

In [7]:
# Display an image in a one percent linear stretch
def display_ls(image: ee.Image, map: geemap.Map, name: str, centered: bool = False) -> None:
    bns = image.bandNames().length().getInfo()
    if bns == 3:
        image = image.rename('B1', 'B2', 'B3')
        pb_99 = ['B1_p99', 'B2_p99', 'B3_p99']
        pb_1 = ['B1_p1', 'B2_p1', 'B3_p1']
        img = ee.Image.rgb(image.select('B1'), image.select('B2'), image.select('B3'))
    else:
        image = image.rename('B1')
        pb_99 = ['B1_p99']
        pb_1 = ['B1_p1']
        img = image.select('B1')
    percentiles = image.reduceRegion(ee.Reducer.percentile([1, 99]), maxPixels=1e11)
    mx = percentiles.values(pb_99)
    if centered:
        mn = ee.Array(mx).multiply(-1).toList()
    else:
        mn = percentiles.values(pb_1)
    map.addLayer(img, {'min': mn, 'max': mx}, name)


## Simple Differences

In [8]:
aoi = ee.FeatureCollection(
    'projects/google/imad_tutorial/landkreis_olpe_aoi').geometry()


In [9]:
def collect(aoi: ee.Geometry, t1a: str, t1b: str, t2a: str, t2b: str) -> Tuple[ee.Image, ee.Image]:
    try:
        im1 = ee.Image( ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
                                .filterBounds(aoi)
                                .filterDate(ee.Date(t1a), ee.Date(t1b))
                                .filter(ee.Filter.contains(rightValue=aoi, leftField='.geo'))
                                .sort('CLOUDY_PIXEL_PERCENTAGE')
                                .first()
                                .clip(aoi) )
        im2 = ee.Image( ee.ImageCollection("COPERNICUS/S2_HARMONIZED")
                                .filterBounds(aoi)
                                .filterDate(ee.Date(t2a), ee.Date(t2b))
                                .filter(ee.Filter.contains(rightValue=aoi, leftField='.geo'))
                                .sort('CLOUDY_PIXEL_PERCENTAGE')
                                .first()
                                .clip(aoi) )
        timestamp = im1.date().format('E MMM dd HH:mm:ss YYYY')
        print(timestamp.getInfo())
        timestamp = im2.date().format('E MMM dd HH:mm:ss YYYY')
        print(timestamp.getInfo())
        return im1, im2
    except Exception as e:
        print('Error: %s'%e)

im1, im2 = collect(aoi, '2021-06-01', '2021-06-30', '2022-06-01', '2022-06-30')

Sun Jun 13 10:36:49 2021
Thu Jun 16 10:46:56 2022


In [23]:
# Interactive Map
M1 = geemap.Map()
M1.centerObject(aoi, 11)

In [24]:
visirbands = ['B2','B3','B4','B8','B11','B12']
visbands = ['B2','B3','B4']

diff = im1.subtract(im2).select(visbands)
# display_ls(im1.select(visbands), M1, 'Image 1')
# display_ls(im2.select(visbands), M1, 'Image 2')
display_ls(diff, M1, 'Difference')


In [25]:
M1

Map(center=[51.08617675675536, 7.976478425048881], controls=(WidgetControl(options=['position', 'transparent_b…

## The MAD Transformation

1) We create a scalar image by taking a linear combination $U = a^{T}X$ of all the bands. 
2) We do the same for the other image ($V = b^{T}Y$) and then perform "similar making".
3) We peform "similar making" by Canonical Correlation Analysis (CCA) (introduced by Harold Hotelling in 1936).

CCA:

$\rho = \frac{cov(U, V)}{\sqrt{var(U)}\sqrt{var(V)}}$

#### Helper Functions for Computing the Covariance and Variance Matrices

In [13]:
def covarw(image: ee.Image, weights = None, scale: int = 20, maxPixels: int = 1e10) -> Tuple[ee.Image, ee.Array]:
    '''Returns the weighted centered image and its weighted covariance matrix'''
    try:
        geometry = image.geometry()
        bandNames = image.bandNames()
        N = bandNames.length()
        if weights is None:
            weights = image.constant(1)
        # Creates a weight for each pixel and adds the new weights to it. It is stored in a ee image.
        weightsImage = image.multiply(ee.Image.constant(0)).add(weights) 
        
        means = image.addBands(weightsImage) \
                    .reduceRegion(ee.Reducer.mean().repeat(N).splitWeights(), # Computes the mean of each band using the weight band as weights
                                  scale = scale,
                                  maxPixels = maxPixels) \
                    .toArray() \
                    .project([1])
                    
        centered: ee.Image = image.toArray().subtract(means)
        # Get first band names. It works because weights has one band and now centered is an array of arrays (1 band)
        B1 = centered.bandNames().get(0) 
        b1 = weights.bandNames().get(0)
        
        nPixels = ee.Number(centered.reduceRegion(ee.Reducer.count(),
                                                  scale = scale,
                                                  maxPixels = maxPixels).get(B1))
        sumWeights = ee.Number(weights.reduceRegion(ee.Reducer.sum(),
                                                    geometry = geometry,
                                                    scale = scale,
                                                    maxPixels = maxPixels).get(b1))
        
        # Computes the centered covariance matrix.
        # It is used to find out how bands vary.
        covw = centered.multiply(weights.sqrt()) \
                    .toArray() \
                    .reduceRegion(ee.Reducer.centeredCovariance(),
                                    geometry = geometry,
                                    scale = scale,
                                    maxPixels = maxPixels) \
                    .get('array')
                    
        covw = ee.Array(covw).multiply(nPixels).divide(sumWeights) # scales it for the number of pixels
        # transforms centered back to image and returns it together with the covariance matrix
        return (centered.arrayFlatten([bandNames]), covw) 
    except Exception as e:
        print('Error: %s'%e)


In [14]:
def corr(cov: ee.Array) -> ee.Array:
    '''Transforms a covariance matrix to a correletation matrix.'''
    # Gets the variances (sigmas) and makes the inverse of a square matrix with them
    sInv = cov.matrixDiagonal().sqrt().matrixToDiag().matrixInverse() 
    # transform: we normalize the corr
    # R = sInv * cov * sInv (math formula, think about covariances and dividing by the right elements)
    corr = sInv.matrixMultiply(cov).matrixMultiply(sInv).getInfo()
    # truncate
    return [list(map(trunc, corr[i])) for i in range(len(corr))]

In [15]:
_, cov = covarw(im1.select(visirbands))
pprint(corr(cov))

[[1.0, 0.952, 0.949, 0.063, 0.647, 0.79],
 [0.952, 0.999, 0.927, 0.291, 0.772, 0.847],
 [0.949, 0.927, 1.0, 0.008, 0.74, 0.893],
 [0.063, 0.291, 0.008, 1.0, 0.486, 0.22],
 [0.647, 0.772, 0.74, 0.486, 1.0, 0.933],
 [0.79, 0.847, 0.893, 0.22, 0.933, 1.0]]


We put 2 images on top of eachother. We look at the top right submatrix to find the covariance of the same band names in both images.

In [16]:
im12 = im1.select(visirbands).addBands(im2.select(visirbands))
_, covar = covarw(im12)
correl = np.array(corr(covar))
print(np.diag(correl[:6, 6:]))

[0.799 0.784 0.682 0.867 0.666 0.632]


#### Solving the Generalized EigenProblem: $C\times X = \lambda\times B\times X$

(This is some linear algebra II eigenvectors equations that I hate that I dont remember :( )

In [17]:
def geneiv(C, B) -> Tuple[ee.Array, ee.Array]:
    try:
        C = ee.Array(C)
        B = ee.Array(B)
        #   Li = choldc(B)^-1
        Li = ee.Array(B.matrixCholeskyDecomposition().get('L')).matrixInverse()
        #   solve symmetric, ordinary eigenproblem Li*C*Li^T*x = lambda*x
        Xa = Li.matrixMultiply(C) \
            .matrixMultiply(Li.matrixTranspose()) \
            .eigen()  # returns both eigenvalues and vectors
            
        #   extracts eingenvalues as a row vector
        lambdas = Xa.slice(1, 0, 1).matrixTranspose() 
        #   eigenvectors as columns
        X = Xa.slice(1, 1).matrixTranspose()
        #  generalized eigenvectors as columns, Li^T*X
        eigenvecs = Li.matrixTranspose().matrixMultiply(X)
        return (lambdas, eigenvecs)
    except Exception as e:
        print('Error: %s'%e)

# The MAD-run Transformation

sum of the squares of the standardized MAD varietes: $\sum_{i=1}^{n} (\frac{M_i}{\sigma_{M_i}})^2$

In [55]:
def mad_run(image1: ee.Image, image2: ee.Image, scale = 20) -> Tuple[ee.Image, ee.Image, ee.Image, ee.Image]:
    try:
        # preprocess the images
        image: ee.Image = image1.addBands(image2)
        region = image.geometry()
        nBands = image.bandNames().length().divide(2)
        centeredImage, covArr = covarw(image, scale=scale)
        bNames = centeredImage.bandNames()
        bNames1 = bNames.slice(0, nBands)
        bNames2 = bNames.slice(nBands)
        centeredImage1 = centeredImage.select(bNames1)
        centeredImage2 = centeredImage.select(bNames2)
        
        s11 = covArr.slice(0, 0, nBands).slice(1, 0, nBands) # Gets the submatrix by selecting columns and rows
        s12 = covArr.slice(0, 0, nBands).slice(1, nBands)
        s21 = covArr.slice(0, nBands).slice(1, 0, nBands)
        s22 = covArr.slice(0, nBands).slice(1, nBands)
        
        # We need to maximize the correlation between bands
        c1 = s12.matrixMultiply(s22.matrixInverse()).matrixMultiply(s21) # normalizes and prepares for CCA
        b1 = s11
        c2 = s21.matrixMultiply(s11.matrixInverse()).matrixMultiply(s12)
        b2 = s22
        
        # solutions of generalized eigenproblem
        #       the eigenvalues  -> relate to the canonical correlations
        #       the eigenvectors -> are the coefficients for the linear combinations that maximize the correlation between the 2 sets
        lambdas, A = geneiv(c1, b1) # 1st image coeff
        _,       B = geneiv(c2, b2) # 2nd image coeff
        
        # represents the canonical correletions between the datasets (images)
        # projects them as a column vector
        rhos = lambdas.sqrt().project(ee.List([1])) 
        
        # MAD variances
        sigma2s = rhos.subtract(1).multiply(-2).toList()  # sigma^2 = 2(1 - rho)    (sigma = variance)
        sigma2s = ee.Image.constant(sigma2s)
        
        # ensure sum of correlations between X and U is positive
        tmp = s11.matrixDiagonal().sqrt()
        ones = tmp.multiply(0).add(1)   # funny trick
        tmp = ones.divide(tmp).matrixToDiag()  # gets the reciprocal of the std deviation
        s = tmp.matrixMultiply(s11).matrixMultiply(A).reduce(ee.Reducer.sum(),[0]).transpose() # normalizes and sums them 
        A = A.matrixMultiply(s.divide(s.abs()).matrixToDiag())   # gets the sign of the sum and multiplies s.t. is positive
        
        # ensure positive correlation between U and V
        tmp = A.transpose().matrixMultiply(s12).matrixMultiply(B).matrixDiagonal() # Computes the correlation between U and V2
        tmp = tmp.divide(tmp.abs()).matrixToDiag()
        B = B.matrixMultiply(tmp)
        
        # canonical and MAD varietes
        centeredImage1Array = centeredImage1.toArray().toArray(1) # specific ee organization
        centeredImage2Array = centeredImage2.toArray().toArray(1)
        # canonical varieties
        U = ee.Image(A.transpose()).matrixMultiply(centeredImage1Array) \
                    .arrayProject([0]) \
                    .arrayFlatten([bNames2])  # projects it to make it 2D then, makes it again an image that makes sense
        V = ee.Image(B.transpose()).matrixMultiply(centeredImage2Array) \
                    .arrayProject([0]) \
                    .arrayFlatten([bNames2])
        MAD = U.subtract(V)
        #   chi square image
        Z = MAD.pow(2) \
                .divide(sigma2s) \
                .reduce(ee.Reducer.sum()) \
                .clip(region) # aggregates all bands per pixel and uses the formula from above
        return (U, V, MAD, Z)
        
    except Exception as e:
        print('Error: %s'%e)

In [56]:
U, V, MAD, Z = mad_run(im1.select(visirbands), im2.select(visirbands))

In [28]:
M2 = geemap.Map()
M2.centerObject(aoi, 11)
# display_ls(im1.select(visbands), M2, 'Image 1')
# display_ls(im2.select(visbands), M2, 'Image 2')
# display_ls(diff, M2, 'Difference')
display_ls(MAD.select(0, 1, 2), M2, 'MAD Image', True)

# M2

Pretty colors but not necessarily easier to interpret. It is not "mittelprachtig" (not so hot)

The canonical variates/correlations (the eigenvalues of CCA) are all *mutually uncorrelated* and are orders by decreasing positive correlation. *Mutually uncorrelated* means that the covariance of any 2 is 0: 

$$cov(U_i, U_j) = 0, i \neq j$$
$$\rho_1 \geq \rho_2 \geq ... \geq \rho_n, \rho_i = corr(U_i, V_i)$$

We can see this below:

In [57]:
_, covar = covarw(U.addBands(V))
correl = np.array(corr(covar))
pprint(correl)
# print s12, s21
print('rho = ', np.diag(correl[:6, 6:]))

array([[ 0.999,  0.001, -0.   ,  0.   , -0.   ,  0.   ,  0.923,  0.   ,
        -0.   ,  0.   , -0.   , -0.   ],
       [ 0.001,  0.999,  0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.864,
         0.   ,  0.   , -0.   , -0.   ],
       [-0.   ,  0.   ,  0.999,  0.   ,  0.   , -0.   , -0.   ,  0.   ,
         0.707, -0.   ,  0.   , -0.   ],
       [ 0.   ,  0.   ,  0.   ,  0.999,  0.   ,  0.   ,  0.   ,  0.   ,
        -0.   ,  0.642,  0.   ,  0.   ],
       [-0.   , -0.   ,  0.   ,  0.   ,  1.   ,  0.   , -0.   , -0.   ,
         0.   ,  0.   ,  0.536,  0.   ],
       [ 0.   , -0.   , -0.   ,  0.   ,  0.   ,  0.999, -0.   , -0.   ,
        -0.   ,  0.   ,  0.   ,  0.369],
       [ 0.923,  0.   , -0.   ,  0.   , -0.   , -0.   ,  1.   ,  0.   ,
        -0.   ,  0.   , -0.   , -0.   ],
       [ 0.   ,  0.864,  0.   ,  0.   , -0.   , -0.   ,  0.   ,  0.999,
         0.   ,  0.   , -0.   , -0.   ],
       [-0.   ,  0.   ,  0.707, -0.   ,  0.   , -0.   , -0.   ,  0.   ,
         1.   , -0.   , 

The MAD varieties are also mutually uncorrelated 
$$cov(M_i, M_j) = cov(U_i - V_i, U_j - V_j) = 0, i \neq j = 1...N$$

and their variances are
$$\sigma^2_{M_i} = var(U_i, V_i) = var(U_i) + var(V_i) - 2cov(U_i, V_i) = 2(1 - \rho_i), i = 1...N.$$

We show:

In [58]:
_, covar = covarw(MAD)
covar = covar.getInfo()
[list(map(trunc, covar[i])) for i in range(len(covar))]

[[0.079, 0.0, -0.0, 0.0, 0.0, 0.0],
 [0.0, 0.14, 0.0, -0.0, -0.0, 0.0],
 [-0.0, 0.0, 0.304, 0.0, -0.0, -0.0],
 [0.0, -0.0, 0.0, 0.372, 0.0, 0.0],
 [0.0, -0.0, -0.0, 0.0, 0.482, 0.0],
 [0.0, 0.0, -0.0, 0.0, 0.0, 0.657]]

# Conclusion

The MAD variates are ordered by their variance. The first MAD variate minimizes the variance as much as possible. The second one minimizes the variance with the added constraint that it must not be correalted with the first, and so on.